# DSG: MultinomialNB

In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import cross_val_score, train_test_split,StratifiedKFold, StratifiedShuffleSplit, KFold
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from lib_DSG import ColumnSelector, DenseTransformer

folder = os.getcwd() ; print folder

/home/arda/Documents/youtube


# Importing data

In [2]:
pd_train = pd.read_csv('./data/train_sample_munged.csv', header=0, escapechar='\\', quotechar='"', low_memory=False, error_bad_lines=False)
pd_test = pd.read_csv('./data/test_sample_munged.csv', header=0, escapechar='\\', quotechar='"', low_memory=False, error_bad_lines=False )

In [3]:
pd_train = pd_train.fillna('')
pd_test = pd_test.fillna('')

In [40]:
(pd_train['duration'].mean())/60.

7.325331312920194

In [42]:
(pd_train['duration'].median()) /60.

4.516666666666667

In [44]:
pd_train['duration'].describe()

count    239225.000000
mean        439.519879
std         553.662756
min          -1.000000
25%         179.000000
50%         271.000000
75%         485.000000
max       15666.000000
Name: duration, dtype: float64

# First model

In [5]:
dico_pattern={'match_lowercase_only':'\\b[a-z]+\\b',
              'match_word':'\\w{1,}',
              'match_word1': '(?u)\\b\\w+\\b',
              'match_3char': '(?u)\\b\\w+\\b\\w+\\b',
              'match_word_punct': '\w+|[,.?!;]',
              'match_NNP': '\\b[A-Z][a-z]+\\b|\\b[A-Z]+\\b',
              'match_punct': "[,.?!;'-]"
             }

stopwords = [u'http', u'com', u'www', u's', u'subscribe'
, u'new', u'like', u'watch', u't', u'2014', u'1'
, u'2013', u'2', u'la', u'en'
, u'world', u'make', u'v', u'check', u'time'
, u'10', u'best', u'3', u'5', u'day', u'y']
tfv_title = TfidfVectorizer(lowercase=True, stop_words=stopwords, token_pattern=dico_pattern["match_word"], 
                      ngram_range=(1, 2), max_df=0.5, min_df=2, max_features=None, 
                      vocabulary=None, binary=True, norm=u'l2', 
                      use_idf=True, smooth_idf=True, sublinear_tf=True)

tfv_desc = TfidfVectorizer(lowercase=True, stop_words='english', token_pattern=dico_pattern["match_word"], 
                      ngram_range=(1, 2), max_df=0.5, min_df=2, max_features=None, 
                      vocabulary=None, binary=True, norm=u'l2', 
                      use_idf=True, smooth_idf=True, sublinear_tf=True)

tfv_topicid = TfidfVectorizer(lowercase=True, stop_words=None, token_pattern=dico_pattern["match_word1"], 
                      ngram_range=(1, 1), max_df=1.0, min_df=2, max_features=None, 
                      vocabulary=None, binary=True, norm=u'l2', 
                      use_idf=True, smooth_idf=True, sublinear_tf=True)

tfv_rel_topic = TfidfVectorizer(lowercase=True, stop_words=None, token_pattern=dico_pattern["match_word1"], 
                      ngram_range=(1, 1), max_df=1.0, min_df=2, max_features=None, 
                      vocabulary=None, binary=True, norm=u'l2', 
                      use_idf=True, smooth_idf=True, sublinear_tf=True)

clf = MultinomialNB(alpha=0.05, fit_prior=True, class_prior=None)

title_pipe = make_pipeline(ColumnSelector(key='title'), tfv_title)
desc_pipe = make_pipeline(ColumnSelector(key='description'), tfv_desc)
topicId_pipe = make_pipeline(ColumnSelector(key=u'topicIds'), tfv_topicid)
reltopicID_pipe = make_pipeline(ColumnSelector(key=u'relevantTopicIds'), tfv_rel_topic)

pipeline = make_union(title_pipe, desc_pipe, topicId_pipe, reltopicID_pipe)
pipeline.transformer_weights=[1, 1, 1, 1]


In [177]:
Y = pd_train[u'video_category_id'].values
X = pipeline.fit_transform(pd_train) ; print X.shape
X_test = pipeline.transform(pd_test)

(239225, 657494)


In [178]:
alphas = np.arange(0.01, 0.2, 0.03)
alphas = [1e-3, 1e-2, 1e-1, 1, 5]
print alphas

[0.001, 0.01, 0.1, 1, 5]


In [179]:
results=[]
for i in alphas:
    clf.alpha = i

    sss = StratifiedShuffleSplit(Y, 10, test_size=0.2, random_state=0)
    scores_sss = cross_val_score(clf, X, Y,scoring='accuracy',cv=sss, n_jobs=-1)

    print ("SSS: acc: %0.4f, std: %0.4f, alpha: %s" %(scores_sss.mean(), scores_sss.std(), i))

SSS: acc: 0.7797, std: 0.0018, alpha: 0.001
SSS: acc: 0.7833, std: 0.0018, alpha: 0.01
SSS: acc: 0.7824, std: 0.0018, alpha: 0.1
SSS: acc: 0.7599, std: 0.0020, alpha: 1
SSS: acc: 0.7291, std: 0.0020, alpha: 5


# OTHER MODEL

In [5]:
pd_train.columns

Index([u'video_category_id', u'title', u'description', u'published_at',
       u'viewCount', u'likeCount', u'dislikeCount', u'favoriteCount',
       u'commentCount', u'duration', u'dimension', u'definition', u'caption',
       u'licensedContent', u'topicIds', u'relevantTopicIds', u'dimension_2d',
       u'dimension_3d', u'definition_hd', u'definition_sd'],
      dtype='object')

In [28]:
sentence = pd_train.apply(lambda r: (r['title'] + r['description'] + r['topicIds'] + r['relevantTopicIds']), axis=1)
sentence_test  = pd_test.apply(lambda r: (r['title'] + r['description'] + r['topicIds'] + r['relevantTopicIds']), axis=1)

In [29]:
stopwords = [u'http', u'com', u'www', u's', u'subscribe' , u'new', u'like', u'watch', u't', u'2014', u'1'
, u'2013', u'2', u'la', u'en', u'world', u'make', u'v', u'check', u'time', u'10', u'best', u'3', u'5', u'day', 
u'y', 'the', 'to', 'a', 'of', 'in', 'floor','for', 'is', 'you','video' , 'this', 'with', 'at', 'it', 'i',
            'that', 'we', 'your' , 'be', 'are', 'about', 'and', 'on', 'what', 'by', 'was', 'http', 'https', 'if',
            'get', 'can', 'up']

dico_pattern={'match_lowercase_only':'\\b[a-z]+\\b',
              'match_word':'\\w{1,}',
              'match_word1': '(?u)\\b\\w+\\b',
              'match_3char': '(?u)\\b\\w+\\b\\w+\\b',
              'match_word_punct': '\w+|[,.?!;]',
              'match_NNP': '\\b[A-Z][a-z]+\\b|\\b[A-Z]+\\b',
              'match_punct': "[,.?!;'-]"
             }
tfv = TfidfVectorizer(lowercase=True, stop_words=stopwords, token_pattern=dico_pattern["match_word1"], 
                      ngram_range=(1, 2), max_df=0.5, min_df=2, max_features=None, 
                      vocabulary=None, binary=True, norm=u'l2', 
                      use_idf=False, smooth_idf=True, sublinear_tf=True)

clf = MultinomialNB(alpha=0.05, fit_prior=True, class_prior=None)

In [30]:
Y = pd_train[u'video_category_id'].values
X = tfv.fit_transform(sentence)
X_test = tfv.transform(sentence_test)
print X.shape
print X_test.shape

(239225, 759913)
(115793, 759913)


In [31]:
numFeat = 40

Features = np.array(tfv.get_feature_names())
sorted_indices = np.argsort(np.array(X.sum(0))[0])[::-1]
rankFeatures = Features[sorted_indices][:numFeat]
print rankFeatures

[u'04rlf' u'm 04rlf' u'04rlf m' u'facebook' u'more' u'bit' u'ly' u'from'
 u'youtube' u'bit ly' u'de' u'my' u'how' u'out' u'twitter' u'here' u'me'
 u'all' u'an' u'our' u'01mw1' u'm 01mw1' u'074ft' u'm 074ft' u'as' u'us'
 u'0k4j' u'm 0k4j' u'02vxn' u'm 02vxn' u'm 0mdxd' u'0mdxd' u'09lgd'
 u'm 09lgd' u'074ft m' u'one' u'01mw1 m' u'do' u'his' u'0k4j m']


In [32]:
alphas = np.arange(0.01, 0.2, 0.03)
alphas = [1e-3, 1e-2, 1e-1, 1, 5]
print alphas

[0.001, 0.01, 0.1, 1, 5]


In [33]:
results=[]
for i in alphas:
    clf.alpha = i
    sss = StratifiedShuffleSplit(Y, 10, test_size=0.2, random_state=0)
    scores_sss = cross_val_score(clf, X, Y,scoring='accuracy',cv=sss, n_jobs=-1)
    print ("SSS: acc: %0.4f, std: %0.4f, alpha: %s" %(scores_sss.mean(), scores_sss.std(), i))

SSS: acc: 0.7847, std: 0.0015, alpha: 0.001
SSS: acc: 0.7844, std: 0.0016, alpha: 0.01
SSS: acc: 0.7733, std: 0.0018, alpha: 0.1
SSS: acc: 0.7354, std: 0.0020, alpha: 1
SSS: acc: 0.6963, std: 0.0019, alpha: 5


# SUBMIT KAGGLE

In [34]:
clf.alpha = 0.01

In [23]:
clf.fit(X,Y)

ValueError: Found arrays with inconsistent numbers of samples: [115793 239225]

In [182]:
y_pred = clf.predict(X_test)

In [183]:
submit = pd.DataFrame(index=None)
submit['id']=pd_test['id']
submit['Pred']=y_pred

In [184]:
submit.to_csv(folder+'/results/0.arda1.csv',sep=';',index=None)

# FINAL DUMP

In [101]:
clf.alpha = 0.01

In [102]:
final_pipe = make_pipeline(pipeline, clf)

In [ ]:
final_pipe.fit